# Langchain integration

### Langchain setup

In [1]:
from langchain.tools import tool
from langchain.agents import AgentType, initialize_agent

In [2]:
import os
from llmstudio.langchain import ChatLLMstudio
from llmstudio.providers import LLM

llm = LLM(provider="openai")
chat_llm = ChatLLMstudio(llm=llm, model="gpt-4o-mini", parameters={"temperature": 0})


### Example 1: Train ticket

In [3]:
# Example 1: Train Ticket

@tool
def get_departure(ticket_number: str):
    """Use this to fetch the departure time of a train"""
    return "12:00 AM"

@tool
def buy_ticket(destination: str):
    """Use this to buy a ticket"""
    return "Bought ticket number 123456"

def assistant(question: str) -> str:
    tools = [get_departure, buy_ticket]
    print(tools)

    # Rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, chat_llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True, debug=True
    )

    response = agent_executor.invoke({"input": question})

    return response



In [4]:
# Test the assistant
assistant('When does my train depart? My ticket is 1234')

[StructuredTool(name='get_departure', description='Use this to fetch the departure time of a train', args_schema=<class 'langchain_core.utils.pydantic.get_departure'>, func=<function get_departure at 0x11ff68900>), StructuredTool(name='buy_ticket', description='Use this to buy a ticket', args_schema=<class 'langchain_core.utils.pydantic.buy_ticket'>, func=<function buy_ticket at 0x11ff68860>)]


> Entering new AgentExecutor chain...


/var/folders/f9/5mgvf90j1wn7w693rp67zms80000gn/T/ipykernel_14929/262021557.py:18: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent_executor = initialize_agent(



Invoking: `get_departure` with `{'ticket_number': '1234'}`


12:00 AMYour train departs at 12:00 AM.

> Finished chain.


{'input': 'When does my train depart? My ticket is 1234',
 'output': 'Your train departs at 12:00 AM.'}

In [5]:
assistant('Buy me a ticket to Madrid and tell the departure time')

[StructuredTool(name='get_departure', description='Use this to fetch the departure time of a train', args_schema=<class 'langchain_core.utils.pydantic.get_departure'>, func=<function get_departure at 0x11ff68900>), StructuredTool(name='buy_ticket', description='Use this to buy a ticket', args_schema=<class 'langchain_core.utils.pydantic.buy_ticket'>, func=<function buy_ticket at 0x11ff68860>)]


> Entering new AgentExecutor chain...

Invoking: `buy_ticket` with `{'destination': 'Madrid'}`


Bought ticket number 123456
Invoking: `get_departure` with `{'ticket_number': '123456'}`


12:00 AMI have bought a ticket to Madrid for you. The departure time is 12:00 AM.

> Finished chain.


{'input': 'Buy me a ticket to Madrid and tell the departure time',
 'output': 'I have bought a ticket to Madrid for you. The departure time is 12:00 AM.'}

### Example 2: Start a party

In [6]:
# Example 2: Start a Party

@tool
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True

@tool
def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters."""
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."

@tool
def dim_lights(brightness: float) -> bool:
    """Dim the lights."""
    print(f"Lights are now set to {brightness:.0%}")
    return True

def assistant(question: str) -> str:
    tools = [power_disco_ball, start_music, dim_lights]
    print(tools)

    # Rebuild agent with new tools
    agent_executor = initialize_agent(
        tools, chat_llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True, debug=True
    )

    response = agent_executor.invoke({"input": question})

    return response


In [7]:
assistant('Turn this into a party!')

[StructuredTool(name='power_disco_ball', description='Powers the spinning disco ball.', args_schema=<class 'langchain_core.utils.pydantic.power_disco_ball'>, func=<function power_disco_ball at 0x126bc7240>), StructuredTool(name='start_music', description='Play some music matching the specified parameters.', args_schema=<class 'langchain_core.utils.pydantic.start_music'>, func=<function start_music at 0x126bc7060>), StructuredTool(name='dim_lights', description='Dim the lights.', args_schema=<class 'langchain_core.utils.pydantic.dim_lights'>, func=<function dim_lights at 0x126bc6fc0>)]


> Entering new AgentExecutor chain...

Invoking: `power_disco_ball` with `{'power': True}`


Disco ball is spinning!
True
Invoking: `start_music` with `{'energetic': True, 'loud': True, 'bpm': 120}`


Starting music! energetic=True loud=True, bpm=120
Never gonna give you up.
Invoking: `dim_lights` with `{'brightness': 30}`


Lights are now set to 3000%
TrueThe party is on! The disco ball is spinning, th

{'input': 'Turn this into a party!',
 'output': 'The party is on! The disco ball is spinning, the music "Never Gonna Give You Up" is playing loudly and energetically, and the lights are dimmed for the perfect atmosphere. Let\'s get this party started! 🎉💃🕺'}

### Example 3: Start a party

* We'll use parallel Tool calls and `bind_tools` as functions will be depecated in v1
* We'll show it work with AzureProvider for reference

In [8]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate

# Azure Setup
from llmstudio.providers import LLM

llm = LLM(provider="azure", 
          api_key=os.environ["AZURE_API_KEY"], 
          api_version=os.environ["AZURE_API_VERSION"],
          api_endpoint=os.environ["AZURE_API_ENDPOINT"])
chat_llm = ChatLLMstudio(llm=llm, model="gpt-4o-mini", parameters={"temperature": 0})


In [9]:

@tool
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True

@tool
def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters."""
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."

@tool
def dim_lights(brightness: float) -> bool:
    """Dim the lights."""
    print(f"Lights are now set to {brightness:.0%}")
    return True


In [10]:

def assistant_new(question: str) -> str:
    tools = [power_disco_ball, start_music, dim_lights]
    print(tools)

    # Rebuild agent with new tools
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful assistant"),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ]
    )
    agent = create_openai_tools_agent(chat_llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools)

    response = agent_executor.invoke({"input": question})

    return response

# Test the new assistant

In [11]:

assistant_new("Turn this into a party!")


[StructuredTool(name='power_disco_ball', description='Powers the spinning disco ball.', args_schema=<class 'langchain_core.utils.pydantic.power_disco_ball'>, func=<function power_disco_ball at 0x126bc5bc0>), StructuredTool(name='start_music', description='Play some music matching the specified parameters.', args_schema=<class 'langchain_core.utils.pydantic.start_music'>, func=<function start_music at 0x126bc7240>), StructuredTool(name='dim_lights', description='Dim the lights.', args_schema=<class 'langchain_core.utils.pydantic.dim_lights'>, func=<function dim_lights at 0x126bc7060>)]
Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=128
Lights are now set to 3000%


{'input': 'Turn this into a party!',
 'output': 'The party is on! 🎉\n\n- The disco ball is spinning.\n- Energetic music is playing: "Never Gonna Give You Up."\n- The lights are dimmed to set the mood.\n\nLet the fun begin! 🕺💃'}

# VertexAI - Gemini models

In [12]:
import os
from llmstudio.langchain import ChatLLMstudio
from llmstudio.providers import LLM

llm = LLM(provider="vertexai", 
          api_key=os.environ["GOOGLE_API_KEY"])
chat_llm = ChatLLMstudio(llm=llm, model = "gemini-1.5-pro-latest", parameters={"temperature":0})

In [13]:
chat_llm.invoke("Olá")

AIMessage(content='Olá! 👋  Como posso te ajudar hoje? 😊 \n', additional_kwargs={}, response_metadata={'token_usage': None, 'model_name': 'gemini-1.5-pro-latest', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-bdc0c8dd-e23e-4436-bee5-ead44e9297ed-0')

In [14]:
from langchain.tools import tool

@tool
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True

@tool
def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters."""
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."

@tool
def dim_lights(brightness: float) -> bool:
    """Dim the lights."""
    print(f"Lights are now set to {brightness:.0%}")
    return True


In [15]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate


def assistant_vertex(question: str) -> str:
    tools = [power_disco_ball, start_music, dim_lights]
    print(tools)

    # Rebuild agent with new tools
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a helpful assistant"),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ]
    )
    agent = create_tool_calling_agent(chat_llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools)

    response = agent_executor.invoke({"input": question})

    return response

In [16]:
assistant_vertex("Turn this into a party!")

[StructuredTool(name='power_disco_ball', description='Powers the spinning disco ball.', args_schema=<class 'langchain_core.utils.pydantic.power_disco_ball'>, func=<function power_disco_ball at 0x126b72980>), StructuredTool(name='start_music', description='Play some music matching the specified parameters.', args_schema=<class 'langchain_core.utils.pydantic.start_music'>, func=<function start_music at 0x1274da520>), StructuredTool(name='dim_lights', description='Dim the lights.', args_schema=<class 'langchain_core.utils.pydantic.dim_lights'>, func=<function dim_lights at 0x1274da3e0>)]
Disco ball is spinning!
Starting music! energetic=True loud=True, bpm=120
Lights are now set to 50%


{'input': 'Turn this into a party!', 'output': "\n\nLet's party! 🎉 🕺 💃 🎶 \n"}